In [13]:
pip install svglib reportlab pdf2image

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
import zipfile

# Set the directory where the zip files are located
zip_files_directory = "C:\\Users\\jason\\Downloads\\testerinos\\images"  # Change this to your source directory
# Set the directory where all extracted files will go
destination_directory = "C:\\Users\\jason\\Downloads\\New folder"  # Change this to your destination folder

# Ensure the destination directory exists
os.makedirs(destination_directory, exist_ok=True)

# Iterate over all zip files in the directory
for zip_filename in os.listdir(zip_files_directory):
    if zip_filename.endswith('.zip'):
        zip_path = os.path.join(zip_files_directory, zip_filename)

        # Extract the reference number from the folder name (assuming "RefNo ABCDEF")
        ref_no = zip_filename.replace("RefNo ", "").replace(".zip", "").strip()

        # Open and extract the zip file
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            for file_info in zip_ref.infolist():
                file_extension = os.path.splitext(file_info.filename)[-1]  # Extract file extension
                new_filename = f"{ref_no}{file_extension}"  # Construct new file name
                extracted_path = os.path.join(destination_directory, new_filename)

                # Ensure no filename conflicts by adding an index if necessary
                counter = 1
                while os.path.exists(extracted_path):
                    new_filename = f"{ref_no}_{counter}{file_extension}"
                    extracted_path = os.path.join(destination_directory, new_filename)
                    counter += 1

                # Extract the file to the destination with the new name
                with zip_ref.open(file_info.filename) as source_file, open(extracted_path, 'wb') as target_file:
                    target_file.write(source_file.read())

print("Extraction and renaming complete.")

Extraction and renaming complete.


In [3]:
import os
from PIL import Image, ExifTags
from svglib.svglib import svg2rlg
from reportlab.graphics import renderPM
from pdf2image import convert_from_path

def fix_image_orientation(img):
    try:
        exif = img._getexif()
        if exif is not None:
            orientation_key = next(
                (key for key, value in ExifTags.TAGS.items() if value == 'Orientation'),
                None
            )
            orientation = exif.get(orientation_key)
            if orientation == 3:
                img = img.rotate(180, expand=True)
            elif orientation == 6:
                img = img.rotate(270, expand=True)
            elif orientation == 8:
                img = img.rotate(90, expand=True)
    except Exception as e:
        print(f"[WARN] Could not handle EXIF orientation: {e}")
    return img

def convert_svg_to_png(input_path):
    output_path = os.path.splitext(input_path)[0] + ".png"
    drawing = svg2rlg(input_path)
    renderPM.drawToFile(drawing, output_path, fmt="PNG")
    os.remove(input_path)
    print(f"[INFO] Converted {input_path} to {output_path} and deleted original file.")

def convert_pdf_to_png(input_path):
    images = convert_from_path(input_path)
    base_name = os.path.splitext(input_path)[0]
    
    if len(images) == 1:
        output_path = f"{base_name}.png"
        images[0].save(output_path, 'PNG')
    else:
        for i, image in enumerate(images):
            output_path = f"{base_name}_{i+1}.png"  # Keeps a simpler numbering format
            image.save(output_path, 'PNG')

    os.remove(input_path)
    print(f"[INFO] Converted {input_path} to PNG files and deleted original file.")

def convert_image_to_png(input_path):
    try:
        with Image.open(input_path) as img:
            img = fix_image_orientation(img)
            output_path = os.path.splitext(input_path)[0] + ".png"
            img.convert('RGBA').save(output_path, 'PNG')
        os.remove(input_path)
        print(f"[INFO] Converted {input_path} to {output_path} and deleted original file.")
    except Exception as e:
        print(f"[ERROR] Error converting {input_path}: {e}")

def convert_images_in_folder(folder, overwrite=True):
    for filename in os.listdir(folder):
        if filename.startswith('.'):
            continue  # Skip hidden files

        input_path = os.path.join(folder, filename)
        if os.path.isdir(input_path):
            continue  # Skip directories

        ext = os.path.splitext(filename)[1].lower()
        
        if ext in ['.jpg', '.jpeg', '.webp', '.raw']:
            convert_image_to_png(input_path)

        elif ext == '.svg':
            convert_svg_to_png(input_path)

        elif ext == '.pdf':
            convert_pdf_to_png(input_path)
        
        elif ext == '.png':
            print(f"[INFO] {filename} is already in PNG format.")
        
        else:
            print(f"[SKIP] Unsupported file type: {filename}")

if __name__ == "__main__":
    folder = "C:\\Users\\jason\\Downloads\\New folder"
    convert_images_in_folder(folder, overwrite=True)

[INFO] Converted C:\Users\jason\Downloads\New folder\67c71cc659b4e05237f88ace.jpg to C:\Users\jason\Downloads\New folder\67c71cc659b4e05237f88ace.png and deleted original file.
[INFO] Converted C:\Users\jason\Downloads\New folder\67c71d1459b4e05237f88dd1.jpg to C:\Users\jason\Downloads\New folder\67c71d1459b4e05237f88dd1.png and deleted original file.
[INFO] Converted C:\Users\jason\Downloads\New folder\67c71d7460e212231c7bb852.jpg to C:\Users\jason\Downloads\New folder\67c71d7460e212231c7bb852.png and deleted original file.
[INFO] 67c722330afde4473c5058ea.png is already in PNG format.
[INFO] Converted C:\Users\jason\Downloads\New folder\67c7cbaf053a9f5e610d07af.jpg to C:\Users\jason\Downloads\New folder\67c7cbaf053a9f5e610d07af.png and deleted original file.
[INFO] Converted C:\Users\jason\Downloads\New folder\67c7cbb78983c5983298d88e.jpg to C:\Users\jason\Downloads\New folder\67c7cbb78983c5983298d88e.png and deleted original file.
[INFO] Converted C:\Users\jason\Downloads\New folder\

In [ ]:
import csv
import json
import os
from collections import defaultdict

# Set file paths and column names manually
csv_file_path = "C:\\Users\\jason\\Downloads\\Book1.csv"  # <-- Set your CSV file location
json_output_path = "C:\\Users\\jason\\Downloads\\test.json"      # <-- Set output JSON file location
name_column = '\ufeffnames'                               # <-- Set the CSV column name for "names"
image_column = 'images'                                   # <-- Set the CSV column name for "images"

data = {
    "img_folder_path": "faces",
    "details": []
}

# Use a dictionary to merge duplicate names and aggregate their images
merged_entries = defaultdict(list)

# Read CSV and build merged dictionary
with open(csv_file_path, mode='r', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        name = row[name_column]
        image_filename = row[image_column] + ".png"  # remove + ".png" if not needed
        merged_entries[name].append(image_filename)

# Build the final JSON structure
for name, images in merged_entries.items():
    entry = {
        "name": name,
        "images": images
    }
    data["details"].append(entry)

# Write to JSON
with open(json_output_path, 'w', encoding='utf-8') as jsonfile:
    json.dump(data, jsonfile, indent=4, ensure_ascii=False)

print(f"✅ Successfully created merged JSON from {csv_file_path} at: {json_output_path}")


✅ Successfully created merged JSON from C:\Fusion\Projects\Real-time-FRS-2.0\simpliFRy\data\AWPS.csv at: C:\Fusion\Projects\Real-time-FRS-2.0\simpliFRy\data\awps.json
